In [16]:
import json
import numpy as np
import boto3
import random
import matplotlib.pyplot as plt
import torch
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

%matplotlib inline


def run_test(dataset, sagemaker_runtime, show_img=False, endpoint_name='sm-mnist-stage'):
    randidx = random.randint(0,9985)
    testdata = {"inputs": dataset.data.view(-1,1,28,28)[randidx:randidx+16].numpy().tolist()}
    response = sagemaker_runtime.invoke_endpoint(
                                EndpointName=endpoint_name, 
                                Body=bytes(json.dumps(testdata), 'utf-8'),
                                ContentType="application/json"
                                )

    result = np.argmax(json.loads(response['Body'].read().decode('utf-8')), axis=1)
    
    if show_img:
        fig, axs = plt.subplots(nrows=1, ncols=16, figsize=(16, 1))
        for i, splt in enumerate(axs):
            splt.imshow(testset.data[randidx+i].numpy())
    return result
    
region = boto3.Session().region_name
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)
endpoint_name='sm-mnist-stage'

mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))])

testset = MNIST("./data", transform=mnist_transform, train=False, download=True)

In [15]:
%%time
for i in range(1000000):
    run_test(testset, sagemaker_runtime, False, 'sm-mnist-prod')

KeyboardInterrupt: 